Note: Before closing, go to Cell > All Output > Clear to keep file size small.

Also make sure this jupyter notebook file is opened using the following command:

```jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000```

### Preprocessing

In [ ]:
# Takes around 7 minutes
import pandas as pd
import nltk
import re
import gensim
import time

all_df = pd.read_csv("train.csv",header = 0)
corpus = all_df["title"]

start = time. time()
stop_list = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.porter.PorterStemmer()
def corpus2docs(corpus):
    # corpus is a object returned by load_corpus that represents a corpus.
    docs1 = []

    for title in corpus:
        doc = nltk.word_tokenize(title)
        docs1.append(doc)
    docs2 = [[w.lower() for w in doc] for doc in docs1]
    docs3 = [[w for w in doc if re.search('^[a-z]+$', w)] for doc in docs2]
    docs4 = [[w for w in doc if w not in stop_list] for doc in docs3]
    docs5 = [[stemmer.stem(w) for w in doc] for doc in docs4]
    return docs5

docs = corpus2docs(corpus)
end = time. time()
print(end - start)
print(docs)

In [ ]:
# Takes around 2 minutes
start = time. time()
dictionary = gensim.corpora.Dictionary(docs)

def docs2vecs(docs, dictionary):
    # docs is a list of documents returned by corpus2docs.
    # dictionary is a gensim.corpora.Dictionary object.
    vecs1 = [dictionary.doc2bow(doc) for doc in docs]
    tfidf = gensim.models.TfidfModel(vecs1)
    vecs2 = [tfidf[vec] for vec in vecs1]
    return vecs2

vecs = docs2vecs(docs, dictionary)
end = time. time()
print(end - start)
print(vecs)
# print(vecs[2])
print(len(docs))

### Clustering

In [ ]:
# Takes around 15 mins
import k_means

start = time. time()
num_tokens = len(dictionary.token2id)
K = 3
clusters = k_means.k_means(vecs, num_tokens, K)

end = time. time()
print(end - start)

print(clusters)

In [ ]:
# Takes around 1.5 hours
start = time. time()
all_df_copy = all_df.copy() # make a copy to avoid modifying original df

for k in range(K):
    N = len(clusters[k])
    for n in range(N):
        all_df_copy.loc[clusters[k][n], str(K) + 'cluster'] = k + 1

all_df_copy.to_csv("clustered3.csv", index=False)
end = time. time()
print(end - start)

To add:

Re-run above code without shuffling (do not shuffle as it affects)

Add checking column for 3 categories label

Repeat code below and run for 58 categories

Check performance

Add some performance calculator (?)

Complete interpreting the clusters section

Consider using translator library for malay words

### Interpreting the Clusters 

Incomplete for both K=3 and K=58

In [ ]:
# Write your code here.

# Take all the file IDs in cluster1
cluster1_fids = [fids[d] for d in cluster1]

# Create an empty list, clust1_words =[]
clust1_words =[]

# Add the words from all files to this list. Use corpus.words and extend method. 
#print(cluster1_fids)
clust1_words = [corpus.words(fid)for fid in cluster1_fids]
clust1_words = corpus.words(cluster1_fids)
#print(clust1_words[0:10])

# Remove the Stopwords from this list
import re
stop_list = nltk.corpus.stopwords.words('english')
clust1_r_stopwords = [w for w in clust1_words if w not in stop_list]
docs2 = [w.lower() for w in clust1_r_stopwords] 
docs3 = [w for w in docs2 if re.search('^[a-z]+$', w)] 
clust1_r_process = [stemmer.stem(w) for w in docs3] 



#Call freq distribution metod and display top 10 words or 20 words
import nltk
fdist= nltk.FreqDist(clust1_r_stopwords)
print(fdist.most_common(10))

fdist= nltk.FreqDist(clust1_r_process)
print(fdist.most_common(10))
